In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold
import lightgbm as lgb

OSError: dlopen(/Users/bytedance/anaconda3/lib/python3.6/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/bytedance/anaconda3/lib/python3.6/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [2]:
train_df=pd.read_csv('../inputs/sensor_train.csv')
test_df=pd.read_csv('../inputs/sensor_test.csv')

In [33]:
def q10(x):
    return x.quantile(0.1)


def q20(x):
    return x.quantile(0.2)


def q30(x):
    return x.quantile(0.3)


def q40(x):
    return x.quantile(0.4)


def q60(x):
    return x.quantile(0.6)


def q70(x):
    return x.quantile(0.7)


def q80(x):
    return x.quantile(0.8)


def q90(x):
    return x.quantile(0.9)

In [17]:
train_df.tail()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18
425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18
425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18
425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18
425358,7291,4917,1.2,-1.7,0.3,1.8,1.4,9.6,18


In [14]:
test_df.head()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg
0,0,71,0.2,1.0,0.6,0.2,5.3,9.2
1,0,150,0.0,1.0,-0.7,0.2,6.0,8.2
2,0,244,-0.2,0.8,-2.4,-0.4,5.3,7.6
3,0,326,2.3,-0.4,-1.6,2.5,4.5,3.4
4,0,409,-0.3,-1.4,3.3,0.2,3.5,12.4


In [42]:
df=train_df

In [25]:
df['acc_x_diff']=df.groupby(['fragment_id','behavior_id'])['acc_x'].diff(1)
df['acc_y_diff']=df.groupby(['fragment_id','behavior_id'])['acc_y'].diff(1)
df['acc_z_diff']=df.groupby(['fragment_id','behavior_id'])['acc_z'].diff(1)
df['acc_xg_diff']=df.groupby(['fragment_id','behavior_id'])['acc_xg'].diff(1)
df['acc_yg_diff']=df.groupby(['fragment_id','behavior_id'])['acc_yg'].diff(1)
df['acc_xg_diff']=df.groupby(['fragment_id','behavior_id'])['acc_xg'].diff(1)
df['acc_zg_diff']=df.groupby(['fragment_id','behavior_id'])['acc_zg'].diff(1)

In [29]:
df.head()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,acc_x_diff,acc_y_diff,acc_z_diff,acc_xg_diff,acc_yg_diff,acc_zg_diff
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,-0.2,0.3,-0.5,-0.2,0.2,-0.4
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,0.0,0.0,0.7,0.5,-0.1,0.6
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,0.0,-0.1,-0.8,-0.1,0.1,-1.8
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,0.0,0.3,1.1,0.1,0.0,1.7


In [30]:
group_df=df.groupby('fragment_id')['behavior_id'].agg({'label':'mean','cnt':'count'}).reset_index()

/Users/bytedance/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [35]:
stat_func=['min','max','mean','median','nunique',q10,q20,q30,q40,q60,q70,q80,q90]
stat_ways = ['min', 'max', 'mean', 'median', 'nunique', 'q_10', 'q_20', 'q_30', 'q_40', 'q_60', 'q_70', 'q_80', 'q_90']
stat_cols=['acc_x','acc_y','acc_z','acc_xg','acc_yg','acc_zg']
group_tmp=df.groupby('fragment_id')[stat_cols].agg(stat_func).reset_index()
group_tmp.columns=['fragment_id']+['{}_{}'.format(i, j) for i in stat_cols for j in stat_ways]

In [36]:
group_df=group_df.merge(group_tmp,on='fragment_id', how='left')

In [37]:
group_df.head()

,fragment_id,label,cnt,acc_x_min,acc_x_max,acc_x_mean,acc_x_median,acc_x_nunique,acc_x_q_10,acc_x_q_20,...,acc_zg_median,acc_zg_nunique,acc_zg_q_10,acc_zg_q_20,acc_zg_q_30,acc_zg_q_40,acc_zg_q_60,acc_zg_q_70,acc_zg_q_80,acc_zg_q_90
0,0,0,57,-0.9,0.4,-0.001754,0.0,10,-0.14,-0.10,...,8.5,14,8.10,8.30,8.4,8.50,8.6,8.60,8.78,8.90
1,1,0,56,-0.5,0.2,-0.025000,0.0,6,-0.10,-0.10,...,8.5,11,8.25,8.40,8.4,8.40,8.5,8.60,8.70,8.85
2,2,0,57,-0.5,0.7,-0.028070,-0.0,12,-0.40,-0.20,...,8.4,18,7.86,8.12,8.3,8.40,8.5,8.60,8.60,8.80
3,3,0,55,-0.4,0.5,0.001818,0.0,10,-0.20,-0.10,...,8.3,16,7.90,8.00,8.1,8.20,8.4,8.50,8.60,8.90
4,4,0,55,-1.3,0.9,0.001818,0.0,14,-0.36,-0.22,...,8.6,27,7.68,8.10,8.2,8.46,8.7,8.88,9.12,9.46


In [48]:
def pre_data(df,is_train=True):
    df['acc_x_diff']=df.groupby('fragment_id')['acc_x'].diff(1)
    df['acc_y_diff']=df.groupby('fragment_id')['acc_y'].diff(1)
    df['acc_z_diff']=df.groupby('fragment_id')['acc_z'].diff(1)
    df['acc_xg_diff']=df.groupby('fragment_id')['acc_xg'].diff(1)
    df['acc_yg_diff']=df.groupby('fragment_id')['acc_yg'].diff(1)
    df['acc_xg_diff']=df.groupby('fragment_id')['acc_xg'].diff(1)
    df['acc_zg_diff']=df.groupby('fragment_id')['acc_zg'].diff(1)
    group_df=pd.DataFrame()
    if is_train:
        group_df=df.groupby('fragment_id')['behavior_id'].agg({'label':'mean','cnt':'count'}).reset_index()
    else:
        group_df['fragment_id']=df['fragment_id'].unique()
    stat_func=['min','max','mean','median','nunique',q10,q20,q30,q40,q60,q70,q80,q90]
    stat_ways = ['min', 'max', 'mean', 'median', 'nunique', 'q_10', 'q_20', 'q_30', 'q_40', 'q_60', 'q_70', 'q_80', 'q_90']
    stat_cols=['acc_x','acc_y','acc_z','acc_xg','acc_yg','acc_zg']
    group_tmp=df.groupby('fragment_id')[stat_cols].agg(stat_func).reset_index()
    group_tmp.columns=['fragment_id']+['{}_{}'.format(i, j) for i in stat_cols for j in stat_ways]
    group_df=group_df.merge(group_tmp,on='fragment_id', how='left')
    
    return group_df

In [54]:
def acc_combo(y_pred,y):
    # 数值ID与行为编码的对应关系
    y_pred=np.argmax(y_pred,axis=0)
    mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[y], mapping[y_pred]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0
    
def build_model(train_,test_,features,label,classes=18,seed=2020):
    train_pred=np.zeros((train_.shape[0],classes))
    test_pred=np.zeros((test_.shape[0],classes))
    n_splits=5
    
    kf=KFold(n_splits=n_splits,shuffle=True,random_state=2020)
    params = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': 'None',
        'num_leaves': 63,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'seed': 1,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'num_class': classes,
        'nthread': 8,
        'verbose': -1,
    }
    for n_fold, (train_idx, valid_idx) in enumerate(kf.split(train_[features]), start=1):
        print('the {} training start ...'.format(n_fold))
        train_x, train_y = train_[features].iloc[train_idx], train_[label].iloc[train_idx]
        valid_x, valid_y = train_[features].iloc[valid_idx], train_[label].iloc[valid_idx]
        
        dtrain = lgb.Dataset(train_x, label=train_y)
        dvalid = lgb.Dataset(valid_x, label=valid_y)

        clf = lgb.train(
            params=params,
            train_set=dtrain,
            num_boost_round=3000,
            valid_sets=[dvalid],
            early_stopping_rounds=100,
            verbose_eval=100,
            feval=acc_combo
        )
        train_pred[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_pred += clf.predict(test_[pred], num_iteration=clf.best_iteration)/kf.n_splits
    print(classification_report(train_[label], np.argmax(train_pred, axis=1), digits=4))

In [49]:
train_df_=pre_data(train_df,is_train=True)
test_df_=pre_data(test_df,is_train=False)

/Users/bytedance/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()


In [50]:
train_df_.columns

Index(['fragment_id', 'label', 'cnt', 'acc_x_min', 'acc_x_max', 'acc_x_mean',
       'acc_x_median', 'acc_x_nunique', 'acc_x_q_10', 'acc_x_q_20',
       'acc_x_q_30', 'acc_x_q_40', 'acc_x_q_60', 'acc_x_q_70', 'acc_x_q_80',
       'acc_x_q_90', 'acc_y_min', 'acc_y_max', 'acc_y_mean', 'acc_y_median',
       'acc_y_nunique', 'acc_y_q_10', 'acc_y_q_20', 'acc_y_q_30', 'acc_y_q_40',
       'acc_y_q_60', 'acc_y_q_70', 'acc_y_q_80', 'acc_y_q_90', 'acc_z_min',
       'acc_z_max', 'acc_z_mean', 'acc_z_median', 'acc_z_nunique',
       'acc_z_q_10', 'acc_z_q_20', 'acc_z_q_30', 'acc_z_q_40', 'acc_z_q_60',
       'acc_z_q_70', 'acc_z_q_80', 'acc_z_q_90', 'acc_xg_min', 'acc_xg_max',
       'acc_xg_mean', 'acc_xg_median', 'acc_xg_nunique', 'acc_xg_q_10',
       'acc_xg_q_20', 'acc_xg_q_30', 'acc_xg_q_40', 'acc_xg_q_60',
       'acc_xg_q_70', 'acc_xg_q_80', 'acc_xg_q_90', 'acc_yg_min', 'acc_yg_max',
       'acc_yg_mean', 'acc_yg_median', 'acc_yg_nunique', 'acc_yg_q_10',
       'acc_yg_q_20', 'acc_yg_q_

In [55]:
use_features=['acc_x_min', 'acc_x_max', 'acc_x_mean',
       'acc_x_median', 'acc_x_nunique', 'acc_x_q_10', 'acc_x_q_20',
       'acc_x_q_30', 'acc_x_q_40', 'acc_x_q_60', 'acc_x_q_70', 'acc_x_q_80',
       'acc_x_q_90', 'acc_y_min', 'acc_y_max', 'acc_y_mean', 'acc_y_median',
       'acc_y_nunique', 'acc_y_q_10', 'acc_y_q_20', 'acc_y_q_30', 'acc_y_q_40',
       'acc_y_q_60', 'acc_y_q_70', 'acc_y_q_80', 'acc_y_q_90', 'acc_z_min',
       'acc_z_max', 'acc_z_mean', 'acc_z_median', 'acc_z_nunique',
       'acc_z_q_10', 'acc_z_q_20', 'acc_z_q_30', 'acc_z_q_40', 'acc_z_q_60',
       'acc_z_q_70', 'acc_z_q_80', 'acc_z_q_90', 'acc_xg_min', 'acc_xg_max',
       'acc_xg_mean', 'acc_xg_median', 'acc_xg_nunique', 'acc_xg_q_10',
       'acc_xg_q_20', 'acc_xg_q_30', 'acc_xg_q_40', 'acc_xg_q_60',
       'acc_xg_q_70', 'acc_xg_q_80', 'acc_xg_q_90', 'acc_yg_min', 'acc_yg_max',
       'acc_yg_mean', 'acc_yg_median', 'acc_yg_nunique', 'acc_yg_q_10',
       'acc_yg_q_20', 'acc_yg_q_30', 'acc_yg_q_40', 'acc_yg_q_60',
       'acc_yg_q_70', 'acc_yg_q_80', 'acc_yg_q_90', 'acc_zg_min', 'acc_zg_max',
       'acc_zg_mean', 'acc_zg_median', 'acc_zg_nunique', 'acc_zg_q_10',
       'acc_zg_q_20', 'acc_zg_q_30', 'acc_zg_q_40', 'acc_zg_q_60',
       'acc_zg_q_70', 'acc_zg_q_80', 'acc_zg_q_90']
build_model(train_df_,test_df_,features=use_features,label='label',classes=18,seed=2020)

the 1 training start ...


NameError: name 'lgb' is not defined